In [ ]:
import getpass
import glob
import copy
import os
import pickle
import random
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio
from typing import List, Any, Callable, Tuple
from pathlib import Path
from random import choice
from tqdm import tqdm


import torch
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils as smp_utils
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split


In [ ]:
segmentation-models-pytorch==0.3.2

In [ ]:
def set_seed(seed):
    os.environ['CUDA_VISIBLE_DEVICES'] = '2022'
    os.environ['PYTHONHASHSEED'] = str(2022)
    np.random.seed(2022)
    random.seed(2022)

    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(2022)

In [ ]:
### utility functions

def clean_string(s: str) -> str:
    """
    extract the tile id and timestamp from a source image folder
    e.g extract 'ID_YYYY_MM' from 'nasa_rwanda_field_boundary_competition_source_train_ID_YYYY_MM'
    """
    s = s.replace(f"{dataset_id}_source_", '').split('_')[1:]
    return '_'.join(s)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform

    Args:
        preprocessing_fn (callbale): data normalization function
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose

    """

    _transform = [
        A.Lambda(image=preprocessing_fn),
        A.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return A.Compose(_transform)


In [ ]:
dataset_id = 'nasa_rwanda_field_boundary_competition'
assets = ['labels']

dataset_id = 'nasa_rwanda_field_boundary_competition'

tr_img_path = f"/kaggle/input/nasaharvest/{dataset_id}/{dataset_id}_source_train/{dataset_id}_source_train"
tr_lbl_path = f"/kaggle/input/nasaharvest/{dataset_id}/{dataset_id}_labels_train/{dataset_id}_labels_train"


train_tiles = [clean_string(s) for s in next(os.walk(tr_img_path))[1]]
tr_tiles_id = list(np.unique([int(tr_id.split("_")[0]) for tr_id in train_tiles]))
tr_tiles_id = [f"{number:02d}" for number in tr_tiles_id]

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [ ]:
class Dataset(BaseDataset):
    """FieldBoundary Dataset. Read images, apply augmentation and preprocessing transformations.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
        augmentation (albumentations.Compose): data transfromation pipeline
            (e.g. flip, scale, etc.)
        preprocessing (albumentations.Compose): data preprocessing
            (e.g. noralization, shape manipulation, etc.)

    """
    def __init__(self, tiles_ids, imgs_dir, masks_dir,
                 data_type='train', augmentation=None,
                 bands=[1,2,3,4],
                ):
        self.ids = tiles_ids
        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.data_type = data_type
        self.augmentation = augmentation
        self.bands = bands

    def __len__(self):
        return len(self.ids)



    def normalize(self, array: np.ndarray):
        array_min, array_max = array.min(), array.max()
        return (array - array_min) / (array_max - array_min)

    def read_image(self,tile):
        bands = []
        for month in ['03', '04', '08', '10', '11', '12']:
            m_bands=[]
            for b in self.bands:
                band_pth = f'{self.imgs_dir}/{dataset_id}_source_{self.data_type}_{tile}_2021_{month}/B0{b}.tif'
                band = rio.open(band_pth)
                band_array = band.read(1).astype(float)
                band_array = band_array.astype(np.float32)
                band_array = self.normalize(band_array)
                m_bands.append(band_array)
            m_bands = np.stack(m_bands, axis=-1)
            bands.append(m_bands)
        x_arr= np.dstack(bands)
        del bands
        return x_arr

    def read_mask(self,tile):

        mask_pth = f'{self.masks_dir}/{dataset_id}_labels_train_{tile}/raster_labels.tif'
        boundary_mask = rio.open(mask_pth)
        boundary_mask = boundary_mask.read(1).astype('float')
        return boundary_mask


    def __getitem__(self, i):

        # read data
        tile = self.ids[i]
        x_arr = self.read_image(tile)

        if self.masks_dir is not None:
            mask = self.read_mask(tile)
            # apply augmentations
            if self.augmentation:
                sample = self.augmentation(image=x_arr,mask=mask)
                x_arr, mask = sample['image'], sample['mask']

        else:
            mask = np.zeros((256,256))

        x_arr = np.transpose(x_arr, [2, 0, 1])
        return x_arr, mask

In [ ]:
import albumentations as A
def get_training_augmentation():
    train_transform = [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.50),

    ]
    return A.Compose(train_transform)

### Segmentation model training

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
ENCODER = 'efficientnet-b3'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# get learning rate
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group['lr']

BATCH_SIZE = 8
LR = 0.01
epochs = 150

In [ ]:
########################## Binary #########################################
set_seed(2022)
gkf = GroupKFold(n_splits=10)
F1_scores = []
idx=1
for tr_idx, val_idx in gkf.split(tr_tiles_id, tr_tiles_id, tr_tiles_id):
    print(f'*************** Fold {idx} *******************')
    tr_ids = list(np.array(tr_tiles_id)[tr_idx])
    val_ids = list(np.array(tr_tiles_id)[val_idx])

    tr_dataset = Dataset(tr_ids, tr_img_path, tr_lbl_path,
                         augmentation=get_training_augmentation(),
                        )

    val_dataset = Dataset(val_ids, tr_img_path, tr_lbl_path,
                          augmentation=None,
                         )

    tr_dataloader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    # create segmentation model with pretrained encoder
    model = smp.Unet(
        encoder_name="efficientnet-b3",
        encoder_weights=ENCODER_WEIGHTS,
        in_channels=48,
        classes=1,
        activation=ACTIVATION,
    )


    # define optomizer
    optim = torch.optim.Adam([
        dict(params=model.parameters(), lr=LR),
    ])
    lr_scheduler = ReduceLROnPlateau(optim, mode='min',
                                     factor=0.5, patience=10)

    # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
    dice_loss = smp.losses.DiceLoss(mode="binary",from_logits=False, smooth=1e-5)
    dice_loss.__name__ = 'Dice_loss'
    metrics = [smp_utils.metrics.Fscore(threshold=0.5)]

    # create epoch runners
    # it is a simple loop of iterating over dataloader`s samples
    train_epoch = smp_utils.train.TrainEpoch(
        model,
        loss=dice_loss,
        metrics=metrics,
        optimizer=optim,
        device=DEVICE,
        verbose=True,
    )

    valid_epoch = smp_utils.train.ValidEpoch(
        model,
        loss=dice_loss,
        metrics=metrics,
        device=DEVICE,
        verbose=True,
    )


    max_score = 0
    max_loss = np.inf
    for i in range(0, epochs):
        current_lr = get_lr(optim)
        print(f'\n ############## Epoch: {i} , lr: {current_lr}')
        train_logs = train_epoch.run(tr_dataloader)
        valid_logs = valid_epoch.run(val_dataloader)


        if valid_logs['Dice_loss'] < max_loss:
            max_loss = valid_logs['Dice_loss']
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), f'./best_model_{idx}.pth')

            print('Model saved!')

        lr_scheduler.step(valid_logs['Dice_loss'])
        if current_lr != get_lr(optim):
            print("Loading best model weights!")
            model.load_state_dict(best_model_wts)
            # create epoch runners
            # it is a simple loop of iterating over dataloader`s samples
            train_epoch = smp_utils.train.TrainEpoch(model, loss=dice_loss,
                                                     metrics=metrics, optimizer=optim,
                                                     device=DEVICE,verbose=True,)

            valid_epoch = smp_utils.train.ValidEpoch(model, loss=dice_loss,
                                                     metrics=metrics,
                                                     device=DEVICE,
                                                     verbose=True,)


    # Validation
    model.load_state_dict(torch.load(f'./best_model_{idx}.pth'))
    valid_epoch = smp_utils.train.ValidEpoch(model,
                                         loss=dice_loss,
                                         metrics=metrics,
                                         device=DEVICE,verbose=False)
    logs = valid_epoch.run(val_dataloader)

    print('f1-score: ', logs['fscore'])

    F1_scores.append(logs['fscore'])
    idx+=1

print(f'Ensemble F1 score : {np.mean(F1_scores)} ')

*************** Fold 1 *******************


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]


 ############## Epoch: 0 , lr: 0.01
valid: 100%|██████████| 6/6 [00:01<00:00,  3.19it/s, Dice_loss - 0.8884, fscore - 0.1116]
Model saved!

 ############## Epoch: 1 , lr: 0.01
valid: 100%|██████████| 6/6 [00:00<00:00,  6.77it/s, Dice_loss - 0.8968, fscore - 0.1032]

 ############## Epoch: 2 , lr: 0.01
valid: 100%|██████████| 6/6 [00:01<00:00,  5.36it/s, Dice_loss - 0.9972, fscore - 0.002825]

 ############## Epoch: 3 , lr: 0.01
valid: 100%|██████████| 6/6 [00:00<00:00,  6.42it/s, Dice_loss - 0.956, fscore - 0.04397]

 ############## Epoch: 4 , lr: 0.01
valid: 100%|██████████| 6/6 [00:01<00:00,  5.52it/s, Dice_loss - 0.8948, fscore - 0.1052]

 ############## Epoch: 5 , lr: 0.01
valid: 100%|██████████| 6/6 [00:01<00:00,  5.14it/s, Dice_loss - 0.893, fscore - 0.107]

 ############## Epoch: 6 , lr: 0.01
valid: 100%|██████████| 6/6 [00:00<00:00,  6.51it/s, Dice_loss - 0.9338, fscore - 0.06617]

 ############## Epoch: 7 , lr: 0.01
valid: 100%|██████████| 6/6 [00:00<00:00,  6.82it/s, Dice_lo

### Data Inferences

In [ ]:
te_img_path = f"/kaggle/input/nasaharvest/{dataset_id}/{dataset_id}_source_test/{dataset_id}_source_test"

test_tiles = [clean_string(s) for s in next(os.walk(te_img_path))[1]]

te_tiles_id = list(np.unique([int(te_id.split("_")[0]) for te_id in test_tiles]))

te_tiles_id = [f"{number:02d}" for number in te_tiles_id]

In [ ]:
test_dataset = Dataset(
    te_tiles_id,
    te_img_path,
    masks_dir=None,
    augmentation=None,
    data_type='test',
)

test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
predictions_dictionary = {}
for i, (xb, _) in enumerate(test_dataloader):
    ens_pr_mask = np.zeros((256,256))
    for m in range(10):
        model.load_state_dict(torch.load(f'./best_model_{m+1}.pth'))

        masks = []
        mask1 = model.predict(xb.cuda())
        mask1 = mask1.squeeze().detach().cpu().numpy()
        masks.append(mask1)
        pr_mask = np.mean(masks, axis=0)
        ens_pr_mask+=mask1
    pr_mask = ens_pr_mask/10
    pr_mask = np.where(pr_mask>=0.5,1, 0 )
    predictions_dictionary.update([(str(te_tiles_id[i]), pd.DataFrame(pr_mask))])

In [ ]:
dfs = []
for key, value in predictions_dictionary.items():
    ftd = value.unstack().reset_index().rename(columns={'level_0': 'row', 'level_1': 'column', 0: 'label'})
    ftd['tile_row_column'] = f'Tile{key}_' + ftd['row'].astype(str) + '_' + ftd['column'].astype(str)
    ftd = ftd[['tile_row_column', 'label']]
    dfs.append(ftd)

sub = pd.concat(dfs)
sub

,tile_row_column,label
0,Tile00_0_0,0
1,Tile00_0_1,0
2,Tile00_0_2,0
3,Tile00_0_3,0
4,Tile00_0_4,0
...,...,...
65531,Tile12_255_251,0
65532,Tile12_255_252,0
65533,Tile12_255_253,0
65534,Tile12_255_254,0


In [ ]:
sub['label'] = sub['label'].astype('int')
sub.to_csv(f"./submission.csv", index = False)
